In [ ]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00


In [ ]:
#imprt libraries
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import math
import re
import random
import string
import time

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
import random
from datasets import load_dataset


In [ ]:
# Load the dataset
dataset = load_dataset('opus100', 'en-es')

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset opus100 downloaded and prepared to /root/.cache/huggingface/datasets/opus100/en-es/0.0.0/256f3196b69901fb0c79810ef468e2c4ed84fbd563719920b1ff1fdc750f7704. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(len(dataset['train']))

1000000


In [ ]:
# Initialize empty lists
en_sentences = []
es_sentences = []
# Iterate over the dataset and append the English and Spanish sentences to their respective lists
# i=0
for example in dataset['train']:
    en_sentences.append(example['translation']['en'].lower())
    es_sentences.append(example['translation']['es'].lower())
    # i += 1
    # if i > 100000:
    #     break

In [ ]:
# Make sure the lists are the same length
assert len(en_sentences) == len(es_sentences)

# Create a list of sentence pairs, separated by a tab character
pairs = [f"{eng}\t{spa}" for eng, spa in zip(en_sentences, es_sentences)]


In [ ]:
# print(pairs[643])

In [ ]:
SOS_token = 0
EOS_token =1

class LangDict:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0:"SOS", 1:"EOS"}
        self.n_words = 2 # SOS + COS = 2

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1



In [ ]:
input_lang = LangDict('english')
output_lang = LangDict('spanish')


In [ ]:
# print("Read %s sentence pairs" % len(pairs))


In [ ]:
Max_length = 10

exn_prefix = ("i am", "i m",
              "he is", "he s",
              "she is", "she s",
              "you are", "you re",
              "we are", "we re",
              "they are", "they re"              
             )

def filterPair(p):
    p1_tok = p[0].split(' ')
    p2_tok = p[1].split(' ')
    
    if len(p1_tok) < Max_length and len(p2_tok) < Max_length:
        return True
    else:
        return False

def BuildfilterdPairs(pairs):
    pairList = list()
    for pair in pairs:
        pair_split = pair.split('\t')
        if filterPair(pair_split)==True:
            pairList.append([pair_split[0], pair_split[1]])
    return pairList




pairs = BuildfilterdPairs(pairs)
print("Trimmed to %s sentence pairs" % len(pairs))
print("Counting words...")



Trimmed to 626030 sentence pairs
Counting words...


In [ ]:
# print(pairs[643])

In [ ]:

for pair in pairs:
    input_lang.addSentence(pair[0])
    output_lang.addSentence(pair[1])
print("Counted words:")
print(input_lang.name, input_lang.n_words)
print(output_lang.name, output_lang.n_words)


Counted words:
english 179800
spanish 227668


In [ ]:
# print("Below is an example of sentence pair:")
# print(random.choice(pairs))        


In [ ]:
# Encoder Setup
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
    
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device = device)


In [ ]:
# Decoder Setup
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim =1)

    
    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden
    
    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device = device)


In [ ]:
## Define helper functions
def indexFromSentence(lang, sentence):
    return [lang.word2index[word]  for word in sentence.split(' ')]

def tensorFromSentence(lang, sentence):
    indexes = indexFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype = torch.long, device = device).view(-1, 1)

def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)


In [ ]:
teacherForcing_r = 0.5

#one training iteration
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, 
         decoder_optimizer, criterion, max_length = Max_length):
    
    #initialize encoder hidden layer weights
    encoder_hidden = encoder.initHidden()
    
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()
    
    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
    
    loss = 0
    
    #encoder
    
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]
    
    decoder_input = torch.tensor([[SOS_token]], device=device)
    decoder_hidden = encoder_hidden
    
    teacherForcing = True if random.random() < teacherForcing_r else False
    #decoder    
    if teacherForcing:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]
            
            
    else:
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
        
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()
        
            loss += criterion(decoder_output, target_tensor[di])
        
            if decoder_input.item() == EOS_token:
                break;
    
    loss.backward()
    
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    return loss.item() / target_length


In [ ]:
#helper function: timers

def asMinutes(s):
    m = math.floor(s/ 60)
    s -= m * 60
    return '%dm %ds' % (m,s)

def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))


In [ ]:
#iterate over training process
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every = 100, learning_rate = 0.01):
    
    start = time.time()
    plot_losses = []
    print_loss_total = 0
    plot_loss_total = 0
    
    encoder_optimizer = optim.SGD(encoder.parameters(), lr = learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr = learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                     for i in range(n_iters)]

    criterion = nn.NLLLoss()
    
    for iter in range(1, n_iters + 1):
        
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        loss = train(input_tensor, target_tensor, encoder,
                    decoder, encoder_optimizer, decoder_optimizer, criterion)
        
        print_loss_total += loss
        plot_loss_total += loss
        
        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter/ n_iters), 
                                         iter, iter/ n_iters  *  100, print_loss_avg))


In [ ]:
def evaluate(encoder, decoder, sentence, max_length=Max_length):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device = device)
        for ei in range(input_length):

            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]
            
            
        decoder_input = torch.tensor([[SOS_token]], device = device)#Start of sentens(SOS)
        
        decoder_hidden  = encoder_hidden
        
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        
        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(decoder_input, decoder_hidden)
            topv, topi, = decoder_output.data.topk(1)
            
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break;
            else:
                decoded_words.append(output_lang.index2word[topi.item()])
            
            decoder_input = topi.squeeze().detach()
        
        return decoded_words, decoder_attentions[:di + 1]


In [ ]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 100000, print_every=5000)


5m 13s (- 99m 7s) (5000 5%) 6.3239
10m 38s (- 95m 42s) (10000 10%) 6.2590
16m 6s (- 91m 15s) (15000 15%) 6.1044
21m 31s (- 86m 5s) (20000 20%) 5.9709
27m 3s (- 81m 9s) (25000 25%) 5.8241
32m 28s (- 75m 47s) (30000 30%) 5.7749
38m 0s (- 70m 35s) (35000 35%) 5.6695
43m 31s (- 65m 17s) (40000 40%) 5.5867
49m 4s (- 59m 59s) (45000 45%) 5.4983
54m 37s (- 54m 37s) (50000 50%) 5.5255
60m 8s (- 49m 12s) (55000 55%) 5.4438
65m 42s (- 43m 48s) (60000 60%) 5.3987
71m 16s (- 38m 22s) (65000 65%) 5.3652
76m 51s (- 32m 56s) (70000 70%) 5.3407
82m 29s (- 27m 29s) (75000 75%) 5.3116
88m 5s (- 22m 1s) (80000 80%) 5.3298
93m 38s (- 16m 31s) (85000 85%) 5.2547
99m 13s (- 11m 1s) (90000 90%) 5.2168
104m 51s (- 5m 31s) (95000 95%) 5.1840
110m 27s (- 0m 0s) (100000 100%) 5.1969


In [ ]:
# Define the paths to save the encoder and decoder models
# encoder_path = 'encoder.pth'
# decoder_path = 'decoder.pth'


In [ ]:
# Save the state dictionaries of the encoder and decoder models
# torch.save(encoder1.state_dict(), encoder_path)
# torch.save(decoder1.state_dict(), decoder_path)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive


In [ ]:
model_save_name = 'encoder_2nd.pt'
path = F"/content/drive/MyDrive/{model_save_name}" 
torch.save(encoder1.state_dict(), path)


In [ ]:
model_save_name = 'decoder_2nd.pt'
path = F"/content/drive/MyDrive/{model_save_name}" 
torch.save(decoder1.state_dict(), path)


In [ ]:
e = 'encoder_2nd.pt'
d = 'decoder_2nd.pt'
encoder_path = F"/content/drive/MyDrive/{e}"
decoder_path = F"/content/drive/MyDrive/{d}"


In [ ]:
# Load the saved state dictionaries into new model instances
encoder2 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder2 = DecoderRNN(hidden_size, output_lang.n_words).to(device)
encoder2.load_state_dict(torch.load(encoder_path))
decoder2.load_state_dict(torch.load(decoder_path))


In [ ]:
def evaluateRandomly(encoder, decoder, n = 10):
    for i in range(n):
        pair = random.choice(pairs)
        print('Sentence from source language: ', pair[0])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('Model generated sentence: ', output_sentence)
        print('Sentence from target language: ', pair[1])
        print('')


In [ ]:
input_eng = input("Enter a sent: ")
output_words, attentions = evaluate(encoder2, decoder2, input_eng)
output_sentence = ' '.join(output_words)


print('Sentence from source language: ', input_eng)
print('Model generated sentence: ', output_sentence)

In [ ]:
%cd /content/drive/My Drive/NLP/PreTrained_models

In [ ]:
# !wget http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.bin.gz
# !gunzip SBW-vectors-300-min5.bin.gz
model = KeyedVectors.load_word2vec_format('SBW-vectors-300-min5.bin', binary=True)
# SBW-vectors-300-min5

In [ ]:
import csv
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
test_es = load_dataset("stsb_multi_mt", name="es", split="test")


In [ ]:
test_en = load_dataset("stsb_multi_mt", name="en", split="dev")


In [ ]:
sentence1 = []
sentence2 = []
sim_scores = []

for length in range (len(test_en)):
  sentence1.append(test_en[length]["sentence1"].lower())
  sim_scores.append(test_en[length]["similarity_score"])


for length in range (len(test_es)):
  sentence2.append(test_es[length]["sentence2"].lower())



In [ ]:
cleaned_sentences2 = []
punctuations = string.punctuation + "¡¿" + "’" + "."  # add any additional punctuation marks that need to be removed
for sentence in sentence2:
    cleaned_sentence = "".join([char for char in sentence if char not in sentence2])
    cleaned_sentence = sentence.translate(str.maketrans("", "", string.punctuation))
    cleaned_sentences2.append(cleaned_sentence)


cleaned_sentences1 = []
cleaned_sentences1 = [''.join(c for c in s if c not in string.punctuation) for s in sentence1]

In [ ]:
from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr     

In [ ]:
sentence3 = []

for sent in cleaned_sentences1[1:2]:
    input_eng = sent
    output_words, attentions = evaluate(encoder2, decoder2, input_eng)
    output_sentence = ' '.join(output_words)
    sentence3.append(output_sentence)
    
sim_scores_broken = sim_scores[1:2]

In [ ]:
# print(output_sentence)
# print(cleaned_sentences2[1:2])
assert len(sentence3) == len(cleaned_sentences1)

# Compute the cosine similarity between corresponding indexed sentences
cos_sim = []
for emb1, emb2 in zip(sentence3, cleaned_sentences1):
    similarity = cosine_similarity(emb1.reshape(1, -1), emb2.reshape(1, -1))[0][0]
    cos_sim.append(similarity)


In [ ]:
# Normalize sim_scores using Min-Max scaling
scaler = MinMaxScaler(feature_range=(0, 1))
sim_scores_norm = scaler.fit_transform(np.array(sim_scores_broken).reshape(-1, 1)).flatten()


# cos_sim and similarity_score are the two lists of similarity scores
# of the same length
assert len(cos_sim) == len(sim_scores_broken)

# Calculate the Pearson correlation coefficient and its p-value
corr, p_value = pearsonr(cos_sim, sim_scores_norm)

print(f"Pearson correlation coefficient: {corr:.3f}")
print(f"P-value: {p_value:.3f}")